In [19]:
! pip install pysus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
## Carregando dimensão CBO
df_cbo = pd.read_csv('/content/drive/My Drive/dm_cbo.csv', sep=';', encoding='latin1')

df_cbo.head()

,CODIGO,TITULO
0,10105,Oficial general da aeronáutica
1,10110,Oficial general do exército
2,10115,Oficial general da marinha
3,10205,Oficial da aeronáutica
4,10210,Oficial do exército


In [22]:
df_cbo['CODIGO'] = df_cbo['CODIGO'].apply(str).str.zfill(6)
df_cbo = df_cbo.rename({'TITULO': 'nm_cbo'}, axis=1)

## SIM

In [23]:
from pysus.online_data.SIM import download

estados = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF',
           'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 
           'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 
           'RO', 'RR', 'SC', 'SP', 'SE', 'TO']
anos = [2015, 2016, 2017, 2018, 2019, 2020]

df_sim = pd.DataFrame()
for ano in anos:
  for uf in estados:
    df = download(uf, ano)
    try:
      df = pd.read_parquet(df[0])
    except KeyError:
      break
    df = df[['DTOBITO', 'IDADE', 'ESTCIV', 'ESC2010', 'TPMORTEOCO', 'OBITOGRAV', 'OBITOPUERP', 'CAUSABAS']]
    df_sim = pd.concat([df_sim, df])

pd.DataFrame(df).head()

,DTOBITO,IDADE,ESTCIV,ESC2010,TPMORTEOCO,OBITOGRAV,OBITOPUERP,CAUSABAS
0,13042020,484,3,1,,,,J989
1,30032020,462,2,2,,,,C349
2,09052020,452,1,1,,,,C900
3,12052020,465,1,1,,,,K746
4,20052020,486,2,1,,,,J438


In [24]:
# Indicador: Mortalidade Materna
##  Afecções do Capítulo XV da CID-10 "Gravidez, Parto e Puerpério", 
##  códigos O00-O99, com exceção das mortes fora do período do puerpério de 42 dias (códigos O96 eO97);
## FONTE: http://tabnet.datasus.gov.br/cgi/sim/Ob_Mu_Id_Fertil_Mat_1996_2012.pdf

## Selecionando as linhas sobre morte materna
df_mort_materna = df_sim[df_sim['CAUSABAS'].str.match('O')]

df_mort_materna = df_mort_materna[(df_mort_materna['CAUSABAS'] != 'O96') &
                                 (df_mort_materna['CAUSABAS'] != 'O97')]

## Transformando a coluna de idade
df_mort_materna['IDADE'] = df_mort_materna['IDADE'].str[1:3].astype(int)

## Transformando a coluna de data
df_mort_materna['DTOBITO'] = pd.to_datetime(df_mort_materna['DTOBITO'], format='%d%m%Y')

## Transformando a coluna de estado civil
df_mort_materna.loc[df_mort_materna["ESTCIV"] == '1', "ESTCIV"] = 'Solteira'
df_mort_materna.loc[df_mort_materna["ESTCIV"] == '2', "ESTCIV"] = 'Casada'
df_mort_materna.loc[df_mort_materna["ESTCIV"] == '3', "ESTCIV"] = 'Viúva'
df_mort_materna.loc[df_mort_materna["ESTCIV"] == '4', "ESTCIV"] = 'Separada judicialmente/divorciada'
df_mort_materna.loc[df_mort_materna["ESTCIV"] == '5', "ESTCIV"] = 'União estável'
df_mort_materna.loc[df_mort_materna["ESTCIV"] == '9', "ESTCIV"] = 'Não informado'

## Transformando a coluna de escolaridade
df_mort_materna.loc[df_mort_materna["ESC2010"] == '0', "ESC2010"] = 'Sem escolaridade'
df_mort_materna.loc[df_mort_materna["ESC2010"] == '1', "ESC2010"] = 'Fundamental I (1ª a 4ª série)'
df_mort_materna.loc[df_mort_materna["ESC2010"] == '2', "ESC2010"] = 'Fundamental II (5ª a 8ª série)'
df_mort_materna.loc[df_mort_materna["ESC2010"] == '3', "ESC2010"] = 'Médio (antigo 2º Grau)'
df_mort_materna.loc[df_mort_materna["ESC2010"] == '4', "ESC2010"] = 'Superior incompleto'
df_mort_materna.loc[df_mort_materna["ESC2010"] == '5', "ESC2010"] = 'Superior completo'
df_mort_materna.loc[df_mort_materna["ESC2010"] == '9', "ESC2010"] = 'Não informado'

## TPMORTEOCO - Situação gestacional ou pósgestacional em que ocorreu o óbito
df_mort_materna.loc[df_mort_materna["TPMORTEOCO"] == '1', "TPMORTEOCO"] = 'na gravidez'
df_mort_materna.loc[df_mort_materna["TPMORTEOCO"] == '2', "TPMORTEOCO"] = 'no parto'
df_mort_materna.loc[df_mort_materna["TPMORTEOCO"] == '3', "TPMORTEOCO"] = 'no abortamento'
df_mort_materna.loc[df_mort_materna["TPMORTEOCO"] == '4', "TPMORTEOCO"] = 'até 42 dias após o término do parto'
df_mort_materna.loc[df_mort_materna["TPMORTEOCO"] == '5', "TPMORTEOCO"] = 'de 43 dias a 1 ano após o término da gestação'
df_mort_materna.loc[df_mort_materna["TPMORTEOCO"] == '8', "TPMORTEOCO"] = 'Não ocorreu nestes períodos'
df_mort_materna.loc[df_mort_materna["TPMORTEOCO"] == '9', "TPMORTEOCO"] = 'Não informado'

## OBITOGRAV - Óbito na gravidez
df_mort_materna.loc[df_mort_materna["OBITOGRAV"] == '1', "OBITOGRAV"] = 'Durante a gravidez'
df_mort_materna.loc[df_mort_materna["OBITOGRAV"] == '2', "OBITOGRAV"] = 'Não morreu durante a gravidez'
df_mort_materna.loc[df_mort_materna["OBITOGRAV"] == '9', "OBITOGRAV"] = 'Não informado'

## OBITOPUERP - Óbito no puerpério
df_mort_materna.loc[df_mort_materna["OBITOPUERP"] == '1', "OBITOPUERP"] = 'Até 42 dias após o parto'
df_mort_materna.loc[df_mort_materna["OBITOPUERP"] == '2', "OBITOPUERP"] = 'De 43 dias a 1 ano após o parto'
df_mort_materna.loc[df_mort_materna["OBITOPUERP"] == '3', "OBITOPUERP"] = 'Não morreu durante o puerpério'
df_mort_materna.loc[df_mort_materna["OBITOPUERP"] == '9', "OBITOPUERP"] = 'Não informado'

## Unificando colunas OBITOGRAV e OBITOPUERP
df_mort_materna["Obito_periodo"] = np.where(df_mort_materna["OBITOGRAV"] == "Não morreu durante a gravidez", df_mort_materna["OBITOPUERP"], df_mort_materna["OBITOGRAV"])
df_mort_materna = df_mort_materna.drop(columns=['OBITOPUERP', 'OBITOGRAV'])
df_mort_materna["Obito_periodo"] = np.where(df_mort_materna["Obito_periodo"] == "Não morreu durante o puerpério", "Morte fora da gravidez e puerpério",df_mort_materna["Obito_periodo"])

## Transformando a coluna de Causas básicas de acordo com CID-10
## Fonte: http://tabnet.datasus.gov.br/cgi/sih/mxcid10lm.htm
df_mort_materna['CAUSABAS'] = df_mort_materna['CAUSABAS'].str[1:3].astype(int)
df_mort_materna.loc[df_mort_materna["CAUSABAS"] == 3, "CAUSA_CID_10"] = 'Aborto espontâneo'
df_mort_materna.loc[df_mort_materna["CAUSABAS"] == 4, "CAUSA_CID_10"] = 'Aborto por razões médicas'
df_mort_materna.loc[((df_mort_materna["CAUSABAS"] >= 0) & (df_mort_materna["CAUSABAS"] <= 2)) | 
                    ((df_mort_materna["CAUSABAS"] >= 5) & (df_mort_materna["CAUSABAS"] <= 8)), "CAUSA_CID_10"] = 'Outras gravidezes que terminam em aborto'

df_mort_materna.loc[((df_mort_materna["CAUSABAS"] >= 10) & (df_mort_materna["CAUSABAS"] <= 16)), "CAUSA_CID_10"] = 'Edema, proteinúria e transtornos hipertensivos na gravidez, parto e puerpério'
df_mort_materna.loc[((df_mort_materna["CAUSABAS"] >= 44) & (df_mort_materna["CAUSABAS"] <= 46)), "CAUSA_CID_10"] = 'Placenta prévia, descolamento prematuro de placenta e hemorragia anteparto'
df_mort_materna.loc[((df_mort_materna["CAUSABAS"] >= 30) & (df_mort_materna["CAUSABAS"] <= 43)) | 
                    ((df_mort_materna["CAUSABAS"] >= 47) & (df_mort_materna["CAUSABAS"] <= 48)), "CAUSA_CID_10"] = 'Outros motivos de assistência à mãe relacionados à cavidade fetal e amniótica, e possíveis problemas de parto'
df_mort_materna.loc[((df_mort_materna["CAUSABAS"] >= 64) & (df_mort_materna["CAUSABAS"] <= 66)), "CAUSA_CID_10"] = 'Trabalho de parto obstruído'
df_mort_materna.loc[df_mort_materna["CAUSABAS"] == 72, "CAUSA_CID_10"] = 'Hemorragia pós-parto'
df_mort_materna.loc[((df_mort_materna["CAUSABAS"] >= 20) & (df_mort_materna["CAUSABAS"] <= 29)) | 
                    ((df_mort_materna["CAUSABAS"] >= 60) & (df_mort_materna["CAUSABAS"] <= 63)) |
                    ((df_mort_materna["CAUSABAS"] >= 67) & (df_mort_materna["CAUSABAS"] <= 71)) | 
                    ((df_mort_materna["CAUSABAS"] >= 73) & (df_mort_materna["CAUSABAS"] <= 75)) | 
                    ((df_mort_materna["CAUSABAS"] >= 81) & (df_mort_materna["CAUSABAS"] <= 84)), "CAUSA_CID_10"] = 'Outras complicações da gravidez e do parto'
df_mort_materna.loc[df_mort_materna["CAUSABAS"] == 80, "CAUSABAS"] = 'Parto único espontâneo'
df_mort_materna.loc[((df_mort_materna["CAUSABAS"] >= 85) & (df_mort_materna["CAUSABAS"] <= 99)), "CAUSA_CID_10"] = 'Complicações predominantemente relacionadas ao puerpério e outras afecções obstétricas, não classificadas em outra parte'
df_mort_materna = df_mort_materna.drop(columns=['CAUSABAS'])

## Criando coluna para contagem
df_mort_materna['n'] = 1


df_mort_materna.head()

,DTOBITO,IDADE,ESTCIV,ESC2010,TPMORTEOCO,Obito_periodo,CAUSA_CID_10,n
318,2020-05-07,30,Solteira,Superior completo,na gravidez,Durante a gravidez,Complicações predominantemente relacionadas ao...,1
2205,2020-01-10,26,Casada,Fundamental II (5ª a 8ª série),até 42 dias após o término do parto,Até 42 dias após o parto,Complicações predominantemente relacionadas ao...,1
4598,2020-04-07,22,Casada,Médio (antigo 2º Grau),no parto,Durante a gravidez,Complicações predominantemente relacionadas ao...,1
5815,2020-01-10,33,Solteira,Médio (antigo 2º Grau),até 42 dias após o término do parto,Até 42 dias após o parto,Hemorragia pós-parto,1
6197,2020-03-17,36,Casada,Médio (antigo 2º Grau),no abortamento,Durante a gravidez,Outras gravidezes que terminam em aborto,1


In [25]:
## Removendo idade errada
df_mort_materna = df_mort_materna.drop(df_mort_materna[df_mort_materna['IDADE'] > 70].index)
df_mort_materna.head()

,DTOBITO,IDADE,ESTCIV,ESC2010,TPMORTEOCO,Obito_periodo,CAUSA_CID_10,n
318,2020-05-07,30,Solteira,Superior completo,na gravidez,Durante a gravidez,Complicações predominantemente relacionadas ao...,1
2205,2020-01-10,26,Casada,Fundamental II (5ª a 8ª série),até 42 dias após o término do parto,Até 42 dias após o parto,Complicações predominantemente relacionadas ao...,1
4598,2020-04-07,22,Casada,Médio (antigo 2º Grau),no parto,Durante a gravidez,Complicações predominantemente relacionadas ao...,1
5815,2020-01-10,33,Solteira,Médio (antigo 2º Grau),até 42 dias após o término do parto,Até 42 dias após o parto,Hemorragia pós-parto,1
6197,2020-03-17,36,Casada,Médio (antigo 2º Grau),no abortamento,Durante a gravidez,Outras gravidezes que terminam em aborto,1


In [26]:
## Salvando CSV
df_mort_materna.to_csv("df_sim.csv",index=False)

## SINASC

In [27]:
from pysus.online_data.sinasc import download

estados = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF',
           'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 
           'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 
           'RO', 'RR', 'SC', 'SP', 'SE', 'TO']

anos = [2015, 2016, 2017, 2018, 2019, 2020]

df_sinasc = pd.DataFrame()
for ano in anos:
  for uf in estados:
    df = download(uf, ano)
    try:
      df = pd.read_parquet(df[0])
    except KeyError:
      break
    df = df[['DTNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'GESTACAO', 'PARTO', 'CONSULTAS']]
    df_sinasc = pd.concat([df_sinasc, df])

df_sinasc.head()

,DTNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,GESTACAO,PARTO,CONSULTAS
0,20082020,30,5,3,999992,01,00,,2,4
1,23082020,32,5,4,141405,02,00,,,1
2,19082020,23,2,4,622020,01,00,5,2,4
3,11102020,40,5,5,322205,00,00,5,2,4
4,27092020,32,5,4,999992,01,00,5,2,4


In [28]:

## Transformando a coluna de estado civil
df_sinasc.loc[df_sinasc["ESTCIVMAE"] == '1', "ESTCIVMAE"] = 'Solteira'
df_sinasc.loc[df_sinasc["ESTCIVMAE"] == '2', "ESTCIVMAE"] = 'Casada'
df_sinasc.loc[df_sinasc["ESTCIVMAE"] == '3', "ESTCIVMAE"] = 'Viúva'
df_sinasc.loc[df_sinasc["ESTCIVMAE"] == '4', "ESTCIVMAE"] = 'Separada judicialmente/divorciada'
df_sinasc.loc[df_sinasc["ESTCIVMAE"] == '5', "ESTCIVMAE"] = 'União estável'
df_sinasc.loc[df_sinasc["ESTCIVMAE"] == '9', "ESTCIVMAE"] = 'Não informado'

## Transformando a coluna de escolaridade
df_sinasc.loc[df_sinasc["ESCMAE"] == '1', "ESCMAE"] = 'Nenhuma'
df_sinasc.loc[df_sinasc["ESCMAE"] == '2', "ESCMAE"] = '1 a 3 anos'
df_sinasc.loc[df_sinasc["ESCMAE"] == '3', "ESCMAE"] = '4 a 7 anos'
df_sinasc.loc[df_sinasc["ESCMAE"] == '4', "ESCMAE"] = '8 a 11 anos '
df_sinasc.loc[df_sinasc["ESCMAE"] == '5', "ESCMAE"] = '12 e mais'
df_sinasc.loc[df_sinasc["ESCMAE"] == '9', "ESCMAE"] = 'Ignorado'

## CODOCUPMAE - Ocupação, conforme a Classificação Brasileira de Ocupações (CBO-2002).
####### TODO: PROCURAR UMA TABELA DIMENSÃO ######
## Baixar daqui http://www.mtecbo.gov.br/cbosite/pages/downloads.jsf
df_sinasc = pd.merge(df_sinasc, df_cbo, left_on = 'CODOCUPMAE', right_on = 'CODIGO', how = 'left').drop('CODIGO', axis = 1).drop('CODOCUPMAE', axis = 1)
df_sinasc["nm_cbo"].fillna("Não informado", inplace=True)

## GESTACAO - Semanas de gestação
df_sinasc.loc[df_sinasc["GESTACAO"] == '1', "GESTACAO"] = 'Menos de 22 semanas'
df_sinasc.loc[df_sinasc["GESTACAO"] == '2', "GESTACAO"] = '22 a 27 semanas'
df_sinasc.loc[df_sinasc["GESTACAO"] == '3', "GESTACAO"] = '28 a 31 semanas'
df_sinasc.loc[df_sinasc["GESTACAO"] == '4', "GESTACAO"] = '32 a 36 semanas'
df_sinasc.loc[df_sinasc["GESTACAO"] == '5', "GESTACAO"] = '37 a 41 semanas'
df_sinasc.loc[df_sinasc["GESTACAO"] == '6', "GESTACAO"] = '42 semanas e mais'
df_sinasc.loc[df_sinasc["GESTACAO"] == '9', "GESTACAO"] = 'Não informado'

## PARTO - Tipo de parto
df_sinasc.loc[df_sinasc["PARTO"] == '1', "PARTO"] = 'Vaginal'
df_sinasc.loc[df_sinasc["PARTO"] == '2', "PARTO"] = 'Cesáreo'
df_sinasc.loc[df_sinasc["PARTO"] == '9', "PARTO"] = 'Não informado'

## CONSULTAS - Número de consultas de pré-natal
df_sinasc.loc[df_sinasc["CONSULTAS"] == '1', "CONSULTAS"] = 'Nenhuma'
df_sinasc.loc[df_sinasc["CONSULTAS"] == '2', "CONSULTAS"] = 'de 1 a 3'
df_sinasc.loc[df_sinasc["CONSULTAS"] == '3', "CONSULTAS"] = 'de 4 a 6'
df_sinasc.loc[df_sinasc["CONSULTAS"] == '4', "CONSULTAS"] = '7 e mais'
df_sinasc.loc[df_sinasc["CONSULTAS"] == '9', "CONSULTAS"] = 'Ignorado'


df_sinasc.head(5)

,DTNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,QTDFILMORT,GESTACAO,PARTO,CONSULTAS,nm_cbo
0,20082020,30,União estável,4 a 7 anos,01,00,,Cesáreo,7 e mais,Não informado
1,23082020,32,União estável,8 a 11 anos,02,00,,,Nenhuma,Comerciante atacadista
2,19082020,23,Casada,8 a 11 anos,01,00,37 a 41 semanas,Cesáreo,7 e mais,Trabalhador volante da agricultura
3,11102020,40,União estável,12 e mais,00,00,37 a 41 semanas,Cesáreo,7 e mais,Técnico de enfermagem
4,27092020,32,União estável,8 a 11 anos,01,00,37 a 41 semanas,Cesáreo,7 e mais,Não informado


In [29]:
df_sinasc['CONSULTAS'].drop_duplicates().head()


0       7 e mais
1        Nenhuma
21      de 4 a 6
37      de 1 a 3
7355    Ignorado
Name: CONSULTAS, dtype: object

In [30]:
## Salvando CSV
df_sinasc.to_csv("df_sinasc.csv",index=False)